In [ ]:
##### HDUL OKUMA ######

import numpy as np
from astropy.io import fits

# Specify the path to your FITS file
fits_file_path = '/Users/nurullah/Desktop/true_PNe/sn/sn/png021.3+02.2.txt_lines1.fits'
# Open the FITS file
with fits.open(fits_file_path) as hdul:
    # Access the data in the FITS file
    primary_hdu = hdul[1]
    data = primary_hdu.data
    
with fits.open(fits_file_path) as hdul:
    hdul.info()

    # Print the transposed data
    print("")
    print(data)

In [ ]:
from astropy.io import fits

# Specify the path to your FITS file
fits_file_path = '/Users/nurullah/Desktop/true_PNe/sn/latexçıktı/neat_fits_çıktı/apng216.fits_fit.fits'

# Open the FITS file
with fits.open(fits_file_path) as hdul:
    # You can access the data in the FITS file
    # For example, printing the information about the FITS file:
    hdul.info()
    
    # You can access different HDU (Header Data Unit) in the FITS file
    # For example, to access the primary HDU (usually the first one):
    primary_hdu = hdul[4]

    # You can access the data and header of the primary HDU
    data = primary_hdu.data
    header = primary_hdu.header

    # You can also print the header and data to see the details
    print("Header:")
    print(repr(header))
    print("Data:")
    print(data)

In [ ]:
#### GDPNE klasöründe bulunan alt klasörlerdeki fits dosyalarını tek bir klasöre kaydetme ###



import os
import shutil

def klasorleri_birlestir(ana_klasor, hedef_klasor):
    if not os.path.exists(hedef_klasor):
        os.makedirs(hedef_klasor)

    for root, dirs, files in os.walk(ana_klasor):
        for file in files:
            if file.endswith('.fits'):
                dosya_yolu = os.path.join(root, file)
                shutil.copy2(dosya_yolu, hedef_klasor)

    print(f'Tüm .fits dosyaları {hedef_klasor} klasörüne kopyalandı.')

if __name__ == "__main__":
    ana_klasor = "/Users/nurullah/Desktop/galaxy_abudance/Uzak_Pc/GPNE/"  # Ana klasörünüzün yolunu belirtin
    cikti_dosyasi = "/Users/nurullah/Desktop/galaxy_abudance/sigma_karşılaştırma/"  # Çıktı dosyasının adını ve yolunu belirtin

    klasorleri_birlestir(ana_klasor, cikti_dosyasi)


In [ ]:
from astropy.io import fits
import pandas as pd
import os

# Dizin içindeki tüm FITS dosyalarını al
fits_directory = '/Users/nurullah/Desktop/galaxy_abudance/Uzak_Pc/sigma_5/bilgiislem/neat_out/'
fits_file_paths = [os.path.join(fits_directory, file) for file in sorted(os.listdir(fits_directory)) if file.endswith('.fits')]

# Excel dosyalarının kaydedileceği dizin
output_directory = '/Users/nurullah/Desktop/galaxy_abudance/Uzak_Pc/sigma_5/bilgiislem/deneme/'
os.makedirs(output_directory, exist_ok=True)

# Her 50 FITS dosyası için ayrı bir Excel dosyası oluştur
batch_size = 50

# Dosya isimlerini saklayacak bir sözlük oluştur
file_names_dict = {}

for i in range(0, len(fits_file_paths), batch_size):
    batch_fits_files = fits_file_paths[i:i + batch_size]

    # Excel dosyasının ismini belirle
    excel_file_name = f'batch_{i // batch_size + 1}.xlsx'
    excel_file_path = os.path.join(output_directory, excel_file_name)

    # Her FITS dosyası için döngü
    with pd.ExcelWriter(excel_file_path, engine='openpyxl') as writer:
        for fits_file_path in batch_fits_files:
            # Open the FITS file
            with fits.open(fits_file_path) as hdul:
                # Access the data and header of the LINES BinTableHDU (hdul[2])
                data = hdul[1].data

                # Convert the FITS data to a pandas DataFrame
                df = pd.DataFrame(data)

                # Sadece belirtilen sütunları seç
                selected_columns = ['WlenRest', 'Flux', 'Uncertainty', 'Ion', 'DereddenedFlux', 'DereddenedFluxLo', 'DereddenedFluxHi']
                df_selected = df[selected_columns]

                # Negatif ve sıfıra eşit olan değerlere sahip satırları filtrele ve sil
                df_selected = df_selected[df_selected['Flux'] > 0.01]

                # Sayfa adını dosya isminin ilk 31 karakteri olarak ayarla
                sheet_name = os.path.basename(fits_file_path)[:31]
                
                # Tam dosya ismini sözlüğe ekle
                file_names_dict[sheet_name] = os.path.basename(fits_file_path)

                # Verileri içeren DataFrame'i yeni bir sayfada kaydet
                df_selected.to_excel(writer, sheet_name=sheet_name, index=False)

    print(f"Data has been successfully exported to {excel_file_path}")

# Dosya isimlerini bir CSV dosyasına kaydedelim
file_names_dict_path = os.path.join(output_directory, 'file_names_dict.csv')
file_names_df = pd.DataFrame(list(file_names_dict.items()), columns=['SheetName', 'FullFileName'])
file_names_df.to_csv(file_names_dict_path, index=False)


In [ ]:
import pandas as pd
import os

# Excel dosyalarının bulunduğu klasörün tam yolu
excel_folder_path = '/Users/nurullah/Desktop/galaxy_abudance/Uzak_Pc/sigma_5/bilgiislem/deneme/'

# Tüm Excel dosyalarını al
excel_files = [os.path.join(excel_folder_path, file) for file in sorted(os.listdir(excel_folder_path)) if file.endswith('.xlsx')]

# Dosya isimleri sözlüğünü oku
file_names_dict_path = os.path.join(excel_folder_path, 'file_names_dict.csv')
file_names_df = pd.read_csv(file_names_dict_path)
file_names_dict = dict(zip(file_names_df['SheetName'], file_names_df['FullFileName']))

# Birleştirilecek verileri tutacak ana DataFrame'i oluştur
main_df = pd.DataFrame()

# Her Excel dosyası için döngü
for excel_file_path in excel_files:
    # Excel dosyasını oku
    df = pd.read_excel(excel_file_path, sheet_name=None)

    # Her sayfa için döngü
    for sheet_name, sheet_data in df.items():
        # 'WlenRest', 'Ion', 'Flux' sütunlarını içeren sayfaları seç
        if all(col in sheet_data.columns for col in ['WlenRest', 'Ion', 'Flux']):
            full_file_name = file_names_dict.get(sheet_name, sheet_name)
            # Eğer ana DataFrame boşsa, direkt ekle
            if main_df.empty:
                main_df = sheet_data[['WlenRest', 'Ion', 'Flux']].copy()
                main_df.rename(columns={'Flux': f'Flux_{full_file_name}'}, inplace=True)
            else:
                # Ana DataFrame'de 'WlenRest' ve 'Ion' sütunlarına göre birleştirme yap
                main_df = main_df.merge(sheet_data[['WlenRest', 'Ion', 'Flux']],
                                        on=['WlenRest', 'Ion'],
                                        how='outer',
                                        suffixes=('', f'_{full_file_name}'))

# NaN değerleri '-' ile değiştir
main_df.fillna('-', inplace=True)

# 'WlenRest' sütununa göre küçükten büyüğe sırala
main_df.sort_values('WlenRest', inplace=True)

# Excel dosyasına ekleyeceğimiz sayfa adı
new_sheet_name = 'Flux_Tablo'

# Yeni bir Excel dosyası oluştur
output_excel_file_path = '/Users/nurullah/Desktop/galaxy_abudance/Uzak_Pc/sigma_5/bilgiislem/deneme/Fluxtablo.xlsx'
with pd.ExcelWriter(output_excel_file_path, engine='openpyxl') as writer:
    main_df.to_excel(writer, sheet_name=new_sheet_name, index=False)

print(f"Merged, sorted data has been successfully added to the sheet {new_sheet_name} in the Excel file.")


In [ ]:
import pandas as pd
import os

# Excel dosyalarının bulunduğu klasörün tam yolu
excel_folder_path = '/Users/nurullah/Desktop/galaxy_abudance/Uzak_Pc/sigma_5/bilgiislem/deneme/'

# Tüm Excel dosyalarını al
excel_files = [os.path.join(excel_folder_path, file) for file in sorted(os.listdir(excel_folder_path)) if file.endswith('.xlsx')]

# Dosya isimleri sözlüğünü oku
file_names_dict_path = os.path.join(excel_folder_path, 'file_names_dict.csv')
file_names_df = pd.read_csv(file_names_dict_path)
file_names_dict = dict(zip(file_names_df['SheetName'], file_names_df['FullFileName']))

# Birleştirilecek verileri tutacak ana DataFrame'i oluştur
main_df = pd.DataFrame()

# Her Excel dosyası için döngü
for excel_file_path in excel_files:
    # Excel dosyasını oku
    df = pd.read_excel(excel_file_path, sheet_name=None)

    # Her sayfa için döngü
    for sheet_name, sheet_data in df.items():
        # 'WlenRest', 'Ion', 'DereddenedFlux' sütunlarını içeren sayfaları seç
        if all(col in sheet_data.columns for col in ['WlenRest', 'Ion', 'DereddenedFlux']):
            full_file_name = file_names_dict.get(sheet_name, sheet_name)
            # Eğer ana DataFrame boşsa, direkt ekle
            if main_df.empty:
                main_df = sheet_data[['WlenRest', 'Ion', 'DereddenedFlux']].copy()
                main_df.rename(columns={'DereddenedFlux': f'DereddenedFlux_{full_file_name}'}, inplace=True)
            else:
                # Ana DataFrame'de 'WlenRest' ve 'Ion' sütunlarına göre birleştirme yap
                main_df = main_df.merge(sheet_data[['WlenRest', 'Ion', 'DereddenedFlux']],
                                        on=['WlenRest', 'Ion'],
                                        how='outer',
                                        suffixes=('', f'_{full_file_name}'))

# NaN değerleri '-' ile değiştir
main_df.fillna('-', inplace=True)

# 'WlenRest' sütununa göre küçükten büyüğe sırala
main_df.sort_values('WlenRest', inplace=True)

# Excel dosyasına ekleyeceğimiz sayfa adı
new_sheet_name = 'DereddenedFlux_Tablo'

# Yeni bir Excel dosyası oluştur
output_excel_file_path = '/Users/nurullah/Desktop/galaxy_abudance/Uzak_Pc/sigma_5/bilgiislem/deneme/DereddebFluxtablo.xlsx'
with pd.ExcelWriter(output_excel_file_path, engine='openpyxl') as writer:
    main_df.to_excel(writer, sheet_name=new_sheet_name, index=False)

print(f"Merged, sorted data has been successfully added to the sheet {new_sheet_name} in the Excel file.")


In [ ]:
import os
import pandas as pd
from astropy.io import fits

# Klasördeki tüm FITS dosyalarını al
fits_folder_path = '/Users/nurullah/Desktop/galaxy_abudance/Uzak_Pc/neat_out/'
fits_files = [file for file in os.listdir(fits_folder_path) if file.endswith('.fits')]

# Boş bir DataFrame oluştur
final_data = pd.DataFrame()

# Her bir FITS dosyasını işle
for fits_file in fits_files:
    fits_path = os.path.join(fits_folder_path, fits_file)

    with fits.open(fits_path) as hdul:
        # İlgili HDU'ya eriş
        results_hdu = hdul['RESULTS']
        
        # İlgilenilen sütunu seç (2. sütun, Python'da indeks 1'e denk gelir)
        selected_data = results_hdu.data.field(1)
        
        # Pandas Series'e dönüştür
        fits_data = pd.Series(selected_data, name=fits_file)
        
        # DataFrame'e ekle
        final_data = pd.concat([final_data, fits_data], axis=1)

# Verileri Excel dosyasına yaz
excel_output_path = '/Users/nurullah/Desktop/galaxy_abudance/Uzak_Pc/sigma_5/bilgiislem/deneme/neat_out_hdul3.xlsx'
final_data.to_excel(excel_output_path, index=False)

print(f"Data from FITS files has been successfully saved to {excel_output_path}.")
